<a href="https://colab.research.google.com/github/archanashokeeniitg/DataMining_ass3/blob/main/DM_Ass3_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-3

FA21: CMPE-255 Sec 48 - Machine Learning

Archana Shokeen (015237378)

San Jose State University

## Dataset
Data Set used - https://www.kaggle.com/achintyatripathi/content-based-news-recomender-system/data


## Data Narrative

This is an open source data set provided by Kaggle.
This is a News Dataset that has articles related to 'India,"World News",'Sports","Education". It covers the various attributes like liks, articles, text, summary, keywords etc.

# Abstract-Results






| Technique  |Library| Wall Time|
| ------------- |-----| ------------- |
| Locality-sensitive hashing| faiss.IndexLSH| 154 ms  |
| Exhaustive Search|faiss.IndexFlatL2 | 96.4 ms  |
|Product Quantization|faiss.IndexPQ|8min 82s|
|Trees and Graphs|AnnoyIndex|3.57 s|
|HNSW|nmslib.init|2min 5s|




In [2]:
!pip install texthero

     |████████████████████████████████| 1.5 MB 7.5 MB/s 
     |████████████████████████████████| 235 kB 64.2 MB/s 
     |████████████████████████████████| 749 kB 54.0 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [3]:
import pandas as pd
import numpy as np
import re

import texthero as hero
from texthero import preprocessing

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


#**Data Loading**

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

df = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/Dataset/DM/result_final.csv')
df.head(5)

Mounted at /content/gdrive


,Unnamed: 0,Unnamed: 0.1,link,text,title,date,keywords,summary,title_summary
0,0,0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07 00:00:00,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
2,2,2,http://techcrunch.com/2020/09/14/youtube-launc...,"As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho...",2020-09-14 00:00:00,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho..."
3,3,3,http://techcrunch.com/2020/09/09/groww-an-inve...,Even as more than 150 million people are using...,"Groww, an investment app for millennials in In...",2020-09-09 00:00:00,"['world', 'yc', 'continuity', 'stocks', 'groww...","YC Continuity, the growth-stage investment fun...","Groww, an investment app for millennials in In..."
4,4,4,http://techcrunch.com/2020/09/15/lanzatech-is-...,As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...,2020-09-15 00:00:00,"['production', 'distributed', 'developing', 's...",As part of the continuing global rollout of La...,LanzaTech is developing a small-scale waste bi...


In [6]:
dataset = df
dataset.head(3)

,Unnamed: 0,Unnamed: 0.1,link,text,title,date,keywords,summary,title_summary
0,0,0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...,2020-09-07 00:00:00,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",Facebook addresses political controversy in In...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
2,2,2,http://techcrunch.com/2020/09/14/youtube-launc...,"As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho...",2020-09-14 00:00:00,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho..."


# Data Wrangling

In [33]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'link', 'text', 'title', 'date',
       'keywords', 'summary', 'title_summary'],
      dtype='object')

In [65]:
df.isnull().sum()

Unnamed: 0         0
Unnamed: 0.1       0
link               0
text               6
title              0
date             499
keywords           3
summary            6
title_summary      6
dtype: int64

In [35]:
dataset = dataset.rename(columns={'Unnamed: 0': 'id'})

In [37]:
dataset.head(3)

,id,Unnamed: 0.1,link,text,title,date,keywords,summary,title_summary
0,0,0,http://techcrunch.com/2020/09/07/vodafone-idea...,"Vodafone Idea, one of the largest telecom oper...",indian telecom giant vodafone idea rebrands ...,2020-09-07 00:00:00,"['rebrands', 'idea', 'vi', 'giant', 'brand', '...","Vodafone Idea, one of the largest telecom oper...",Indian telecom giant Vodafone Idea rebrands as...
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",facebook addresses political controversy indi...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
2,2,2,http://techcrunch.com/2020/09/14/youtube-launc...,"As TikTok’s fate in the U.S. remains murky, Yo...",youtube launches tiktok rival youtube shorts...,2020-09-14 00:00:00,"['rival', 'video', 'feature', 'access', 'youtu...","As TikTok’s fate in the U.S. remains murky, Yo...","YouTube launches its TikTok rival, YouTube Sho..."


##**Data Cleaning**

In [68]:
custom_pipeline = [preprocessing.fillna,
                   preprocessing.remove_whitespace,
                   preprocessing.remove_diacritics,
                   preprocessing.remove_punctuation,
                   preprocessing.remove_stopwords,
                   preprocessing.remove_digits,
                   preprocessing.lowercase
                  ]


In [67]:
dataset['title'] = hero.clean(dataset['title'], custom_pipeline)
dataset['title'][0]

'indian telecom giant vodafone idea rebrands vi techcrunch'

##Data Vectorization 

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word',ngram_range=(1, 3),min_df=0,stop_words='english')
X_tfidf = tfidf.fit_transform(dataset['title'])
X_tfidf

<2190x24295 sparse matrix of type '<class 'numpy.float64'>'
	with 42144 stored elements in Compressed Sparse Row format>

In [15]:
vector = X_tfidf.toarray()
vector[0].shape[0]

24295

#**Approximate Nearest Neighbor Search Various Techniques**

In [22]:
!apt install libomp-dev
!python -m pip install --upgrade faiss faiss-gpu
import faiss

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (360 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 155222 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [17]:
!pip install faiss

     |████████████████████████████████| 4.7 MB 5.8 MB/s 


In [23]:
import faiss    

##**Locality-Sensitive Hashing**

In [69]:
class LSHIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, num_bits=8):
        self.index = faiss.IndexLSH(self.dimention,num_bits)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [70]:
%%time
lsh_index = LSHIndex(vector, dataset['id'])
lsh_index.build()

CPU times: user 180 ms, sys: 1e+03 µs, total: 181 ms
Wall time: 154 ms


In [39]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = lsh_index.query(np.array([vector[query_index]]).astype('float32'))
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 1 are:


,id,Unnamed: 0.1,link,text,title,date,keywords,summary,title_summary
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",facebook addresses political controversy indi...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
27,27,27,https://mashable.com/article/charli-damelio-tr...,"Major TikTok creators are joining Triller, inc...",charli d amelio joins triller tiktok rival app,2020-09-15 21:28:11+00:00,"['rival', 'charli', 'dixie', 'damelio', 'tikto...","Major TikTok creators are joining Triller, inc...","Charli D'Amelio joins Triller, TikTok's rival ..."
194,194,194,https://in.reuters.com/article/koninklijke-bos...,(Refiles to correct currency conversion)\n\nAu...,refile boskalis sees steady full year profit ...,2020-08-20 05:33:29+00:00,"['euros', 'profit', 'fullyear', 'trybuś', 'buy...",(Refiles to correct currency conversion)Aug 20...,"REFILE-Boskalis sees steady full-year profit, ..."
209,209,209,http://techcrunch.com/2020/09/02/dear-sophie-c...,"Here’s another edition of “Dear Sophie,” the a...",dear sophie can sponsor h 1b university res...,2020-09-02 00:00:00,"['candidate', 'petition', 'sophie', 'universit...",Dear Sophie:Our company is considering sponsor...,Dear Sophie: Can we sponsor an H-1B university...
248,248,248,https://in.reuters.com/article/us-twitter-cybe...,Slideshow ( 2 images )\n\nBENGALURU (Reuters) ...,twitter confirms account pm modi personal we...,2020-09-03 01:33:23+00:00,"['modis', 'twitter', 'taken', 'hacked', 'confi...",Slideshow ( 2 images )BENGALURU (Reuters) - Tw...,Twitter confirms account of PM Modi's personal...
255,255,255,https://in.reuters.com/article/cricket-t20-sco...,Sep 1 (OPTA) - Scoreboard at close of play of ...,pakistan england scoreboard,2020-09-01 20:41:48+00:00,"['scoreboard', 'pakistan', 'tom', 'jordan', 'i...",Sep 1 (OPTA) - Scoreboard at close of play of ...,Pakistan in England 2020 Scoreboard Sep 1 (OPT...
285,285,285,https://in.reuters.com/article/soccer-uefanati...,REYKJAVIK (Reuters) - England’s Raheem Sterlin...,england scrape win iceland late penalty drama,2020-09-05 21:41:55+00:00,"['spot', 'iceland', 'goal', 'late', 'sterling'...","However, Birkir Bjarnason fired his penalty hi...",England scrape win in Iceland after late penal...
374,374,374,https://in.reuters.com/article/global-race-pro...,(Reuters) - Police arrested more than 60 peopl...,police arrest louisville anti racism protests,2020-08-26 07:27:31+00:00,"['burst', 'black', 'antiracism', '64', 'protes...",Sixty-four people were charged with obstructio...,Police arrest 64 in Louisville anti-racism pro...
500,500,500,https://vitals.lifehacker.com/take-a-free-cour...,Graphic : Andrii Vodolazhskyi ( Shutterstock )...,take free course covid from mit,NaN,"['pandemic', 'course', 'pt', 'mit', 'lectures'...","ET (8:30 a.m. PT ), MIT is offering a course o...",Take a Free Course on COVID-19 From MIT Graphi...
612,612,612,https://www.theverge.com/2020/9/4/21422653/ube...,Uber announce today that it will soon allow us...,uber launches car rentals uk maximum social...,2020-09-04 00:00:00,"['pandemic', 'cars', 'hours', 'social', 'uk', ...",Uber announce today that it will soon allow us...,Uber launches car rentals in the UK for maximu...


##**Exhaustive Search**

In [40]:
class BruteForceIndex():
    def __init__(self, vectors, labels):
        self.vectors = vectors.astype('float32')
        self.labels = labels
        self.index = faiss.IndexFlatL2(vectors[0].shape[0])
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

In [41]:
%%time
es_index = BruteForceIndex(vector, dataset['id'])

CPU times: user 96.4 ms, sys: 397 ms, total: 494 ms
Wall time: 515 ms


In [42]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = es_index.query(np.array([vector[query_index]]).astype('float32'))
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 1 are:


,id,Unnamed: 0.1,link,text,title,date,keywords,summary,title_summary
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",facebook addresses political controversy indi...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
75,75,75,https://www.reuters.com/article/us-health-coro...,FILE PHOTO: A test tube labelled with the Vacc...,serum institute gets approval resume india tr...,2020-09-16 06:29:19+00:00,"['trial', 'covid', 'trials', 'respond', 'resum...",REUTERS/Dado Ruvic/Illustration/File PhotoMUMB...,Serum Institute gets approval to resume India ...
211,211,211,http://techcrunch.com/2020/09/03/facebook-to-b...,We’re now 61 days away from the U.S. president...,facebook block new political ads week nov ...,2020-09-03 00:00:00,"['nov', 'voting', 'information', 'elections', ...",CEO Mark Zuckerberg today announced a series o...,Facebook to block new political ads 1 week bef...
223,223,223,https://www.cnn.com/2020/09/04/asia/sri-lanka-...,NaN,,NaN,NaN,NaN,NaN
231,231,231,https://in.reuters.com/article/india-morningca...,Sept 2 (Reuters) - To access a PDF version of ...,morning news call india september,2020-09-02 03:17:16+00:00,"['coronavirus', 'global', 'manufacturing', 'da...",• India's coronavirus surge eases slightly as ...,"Morning News Call - India, September 2 Sept 2 ..."
374,374,374,https://in.reuters.com/article/global-race-pro...,(Reuters) - Police arrested more than 60 peopl...,police arrest louisville anti racism protests,2020-08-26 07:27:31+00:00,"['burst', 'black', 'antiracism', '64', 'protes...",Sixty-four people were charged with obstructio...,Police arrest 64 in Louisville anti-racism pro...
619,619,619,http://techcrunch.com/2020/09/03/explore-micro...,"Micromobility, like many other industries, has...",explore micromobility next opportunities tc ...,2020-09-03 00:00:00,"['opportunities', 'operators', 'innovation', '...",And despite the over-saturation of companies i...,Explore micromobility’s next opportunities at ...
629,629,629,http://techcrunch.com/2020/09/03/facebook-to-b...,We’re now 61 days away from the U.S. president...,facebook block new political ads week nov ...,2020-09-03 00:00:00,"['nov', 'voting', 'information', 'elections', ...",CEO Mark Zuckerberg today announced a series o...,Facebook to block new political ads 1 week bef...
737,737,737,http://techcrunch.com/2020/09/07/humanforest-g...,"HumanForest, a dockless, shared and ad-support...",humanforest gets 3m cabify founders other...,2020-09-07 00:00:00,"['grow', 'ride', 'humanforest', 'gets', 'londo...","HumanForest, a dockless, shared and ad-support...",HumanForest gets $2.3M from Cabify founders an...
901,1,1,https://arstechnica.com/science/2020/08/corona...,The pandemic coronavirus has made its way onto...,coronavirus creeps us mink farms unusually ...,NaN,"['creeps', 'numbers', 'cases', 'reported', 'to...",The pandemic coronavirus has made its way onto...,"Coronavirus creeps into US mink farms, “unusua..."


##**Product Quantization**

In [43]:
class PQIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors.shape[1]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_partition=8, search_in_x_partitions=2, subvector_size=8):
        m = 3
        n_bits = 8
        self.index = faiss.IndexPQ (self.dimention, m, n_bits)
        self.index.train(self.vectors)
        self.index.add(self.vectors)
        
    def query(self, vectors, k=10):
        distances, indices = self.index.search(vectors, k) 
        return [self.labels[i] for i in indices[0]]

The number of bits n_bits (bits allocated per subquantizer) must be equal to 8, 12 or 16. The dimension d should be a multiple of m (number of subquantizers). Therefore, we need to apply batch here since the vector dimension is 49811, which is a prime number.

In [44]:
batch_vector = []
for idx, arr in enumerate(vector):
    batch_vector.append(list(np.append(arr, 0.0)))

In [60]:
%%time
batch_vector = np.array(batch_vector)
pq_index = PQIndex(batch_vector, dataset['id'])
pq_index.build()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.82 µs


In [62]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
# items_list = pq_index.query(np.array([batch_vector[query_index]]).astype('float32'))
# dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 1 are:


##**Trees and Graphs**

In [48]:
!pip install annoy

     |████████████████████████████████| 646 kB 5.3 MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=391666 sha256=23e7b9c33c41b2855fc58756f6ee979c36c58a5a34e0c8c985b55684e35dc75c
  Stored in directory: /root/.cache/pip/wheels/4f/e8/1e/7cc9ebbfa87a3b9f8ba79408d4d31831d67eea918b679a4c07
Successfully built annoy


In [49]:
import annoy

In [50]:
class TreeGraphIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels


    def build(self, number_of_trees=5):
        self.index = annoy.AnnoyIndex(self.dimention, metric='angular')
        for i, vec in enumerate(self.vectors):
            self.index.add_item(i, vec.tolist())
        self.index.build(number_of_trees)
        
    def query(self, vector, k=10):
        indices = self.index.get_nns_by_vector(vector.tolist(), k)
        return [self.labels[i] for i in indices]

In [51]:
%%time
tg_index = TreeGraphIndex(vector, dataset['id'])
tg_index.build()

CPU times: user 3.33 s, sys: 233 ms, total: 3.56 s
Wall time: 3.57 s


In [52]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = tg_index.query(vector[query_index])
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 1 are:


,id,Unnamed: 0.1,link,text,title,date,keywords,summary,title_summary
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",facebook addresses political controversy indi...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
75,75,75,https://www.reuters.com/article/us-health-coro...,FILE PHOTO: A test tube labelled with the Vacc...,serum institute gets approval resume india tr...,2020-09-16 06:29:19+00:00,"['trial', 'covid', 'trials', 'respond', 'resum...",REUTERS/Dado Ruvic/Illustration/File PhotoMUMB...,Serum Institute gets approval to resume India ...
211,211,211,http://techcrunch.com/2020/09/03/facebook-to-b...,We’re now 61 days away from the U.S. president...,facebook block new political ads week nov ...,2020-09-03 00:00:00,"['nov', 'voting', 'information', 'elections', ...",CEO Mark Zuckerberg today announced a series o...,Facebook to block new political ads 1 week bef...
374,374,374,https://in.reuters.com/article/global-race-pro...,(Reuters) - Police arrested more than 60 peopl...,police arrest louisville anti racism protests,2020-08-26 07:27:31+00:00,"['burst', 'black', 'antiracism', '64', 'protes...",Sixty-four people were charged with obstructio...,Police arrest 64 in Louisville anti-racism pro...
619,619,619,http://techcrunch.com/2020/09/03/explore-micro...,"Micromobility, like many other industries, has...",explore micromobility next opportunities tc ...,2020-09-03 00:00:00,"['opportunities', 'operators', 'innovation', '...",And despite the over-saturation of companies i...,Explore micromobility’s next opportunities at ...
629,629,629,http://techcrunch.com/2020/09/03/facebook-to-b...,We’re now 61 days away from the U.S. president...,facebook block new political ads week nov ...,2020-09-03 00:00:00,"['nov', 'voting', 'information', 'elections', ...",CEO Mark Zuckerberg today announced a series o...,Facebook to block new political ads 1 week bef...
737,737,737,http://techcrunch.com/2020/09/07/humanforest-g...,"HumanForest, a dockless, shared and ad-support...",humanforest gets 3m cabify founders other...,2020-09-07 00:00:00,"['grow', 'ride', 'humanforest', 'gets', 'londo...","HumanForest, a dockless, shared and ad-support...",HumanForest gets $2.3M from Cabify founders an...
837,837,837,http://techcrunch.com/2020/09/11/10-poland-bas...,Poland is becoming an important European tech ...,poland based investors discuss trends oppor...,2020-09-11 00:00:00,"['opportunities', 'investors', 'companies', 't...","Meanwhile, the startup ecosystem has, in recen...","10 Poland-based investors discuss trends, oppo..."
901,1,1,https://arstechnica.com/science/2020/08/corona...,The pandemic coronavirus has made its way onto...,coronavirus creeps us mink farms unusually ...,NaN,"['creeps', 'numbers', 'cases', 'reported', 'to...",The pandemic coronavirus has made its way onto...,"Coronavirus creeps into US mink farms, “unusua..."
975,75,75,https://abcnews.go.com/Politics/timeline-years...,Timeline: This year’s political fight over the...,timeline this year political fight postal ...,NaN,"['mail', 'trump', 'voting', 'ballots', 'voters...",Timeline: This year’s political fight over the...,Timeline: This year’s political fight over the...


##**Hierarchical Navigable Small Worlds (HNSW)**

In [54]:
!pip install nmslib

     |████████████████████████████████| 13.5 MB 5.0 MB/s 
     |████████████████████████████████| 188 kB 50.1 MB/s 


In [55]:
import nmslib

In [56]:
class HNSWIndex():
    def __init__(self, vectors, labels):
        self.dimention = vectors[0].shape[0]
        self.vectors = vectors.astype('float32')
        self.labels = labels

    def build(self):
        self.index = nmslib.init(method='hnsw', space='cosinesimil')
        self.index.addDataPointBatch(self.vectors)
        self.index.createIndex({'post': 2})
        
    def query(self, vector, k=10):
        indices = self.index.knnQuery(vector, k=k)
        return [self.labels[i] for i in indices[0]]

In [57]:
%%time
hnsw_index = HNSWIndex(vector, dataset['id'])
hnsw_index.build()

CPU times: user 3min 58s, sys: 631 ms, total: 3min 58s
Wall time: 2min 5s


In [58]:
query_index = 1
print(f"The most simillar to item {dataset['id'][query_index]} are:")
items_list = hnsw_index.query(vector[query_index])
dataset.loc[dataset['id'].isin(items_list)]

The most simillar to item 1 are:


,id,Unnamed: 0.1,link,text,title,date,keywords,summary,title_summary
1,1,1,http://techcrunch.com/2020/09/16/facebook-addr...,"At the beginning of the previous decade, Faceb...",facebook addresses political controversy indi...,2020-09-16 00:00:00,"['opportunities', 'whatsapp', 'controversy', '...",Politicians in the country today heavily rely ...,Facebook addresses political controversy in In...
75,75,75,https://www.reuters.com/article/us-health-coro...,FILE PHOTO: A test tube labelled with the Vacc...,serum institute gets approval resume india tr...,2020-09-16 06:29:19+00:00,"['trial', 'covid', 'trials', 'respond', 'resum...",REUTERS/Dado Ruvic/Illustration/File PhotoMUMB...,Serum Institute gets approval to resume India ...
211,211,211,http://techcrunch.com/2020/09/03/facebook-to-b...,We’re now 61 days away from the U.S. president...,facebook block new political ads week nov ...,2020-09-03 00:00:00,"['nov', 'voting', 'information', 'elections', ...",CEO Mark Zuckerberg today announced a series o...,Facebook to block new political ads 1 week bef...
374,374,374,https://in.reuters.com/article/global-race-pro...,(Reuters) - Police arrested more than 60 peopl...,police arrest louisville anti racism protests,2020-08-26 07:27:31+00:00,"['burst', 'black', 'antiracism', '64', 'protes...",Sixty-four people were charged with obstructio...,Police arrest 64 in Louisville anti-racism pro...
619,619,619,http://techcrunch.com/2020/09/03/explore-micro...,"Micromobility, like many other industries, has...",explore micromobility next opportunities tc ...,2020-09-03 00:00:00,"['opportunities', 'operators', 'innovation', '...",And despite the over-saturation of companies i...,Explore micromobility’s next opportunities at ...
629,629,629,http://techcrunch.com/2020/09/03/facebook-to-b...,We’re now 61 days away from the U.S. president...,facebook block new political ads week nov ...,2020-09-03 00:00:00,"['nov', 'voting', 'information', 'elections', ...",CEO Mark Zuckerberg today announced a series o...,Facebook to block new political ads 1 week bef...
737,737,737,http://techcrunch.com/2020/09/07/humanforest-g...,"HumanForest, a dockless, shared and ad-support...",humanforest gets 3m cabify founders other...,2020-09-07 00:00:00,"['grow', 'ride', 'humanforest', 'gets', 'londo...","HumanForest, a dockless, shared and ad-support...",HumanForest gets $2.3M from Cabify founders an...
837,837,837,http://techcrunch.com/2020/09/11/10-poland-bas...,Poland is becoming an important European tech ...,poland based investors discuss trends oppor...,2020-09-11 00:00:00,"['opportunities', 'investors', 'companies', 't...","Meanwhile, the startup ecosystem has, in recen...","10 Poland-based investors discuss trends, oppo..."
901,1,1,https://arstechnica.com/science/2020/08/corona...,The pandemic coronavirus has made its way onto...,coronavirus creeps us mink farms unusually ...,NaN,"['creeps', 'numbers', 'cases', 'reported', 'to...",The pandemic coronavirus has made its way onto...,"Coronavirus creeps into US mink farms, “unusua..."
975,75,75,https://abcnews.go.com/Politics/timeline-years...,Timeline: This year’s political fight over the...,timeline this year political fight postal ...,NaN,"['mail', 'trump', 'voting', 'ballots', 'voters...",Timeline: This year’s political fight over the...,Timeline: This year’s political fight over the...


#**Summary**

For this dataset the fastest proven technique was Exhaustive Search and the slowest was Product Quantization





**References**: 

https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6


https://github.com/eyaltrabelsi/my-notebooks/tree/master/Lectures/search_in_practice-approximate_nearest_neighbors